In [0]:
import os, matplotlib, pickle, six
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.misc
import pandas as pd
from imgaug import augmenters as iaa
import keras
from keras.utils.vis_utils import plot_model
from keras.models import Model, load_model
from keras.layers import Input, Activation, Dense, Flatten, Dropout, GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D, SeparableConv2D
from keras.layers.merge import add
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
from imageio import imread
from keras.callbacks import *
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping, ModelCheckpoint

batch_size = 256
nb_class = 200
nb_epoch = 300
img_per_class = 500

# input image dimensions
img_height, img_width = 64, 64
channels = 3

In [0]:
!wget 'http://cs231n.stanford.edu/tiny-imagenet-200.zip'
!unzip -qq '/tiny-imagenet-200.zip'

In [0]:
# Image Augmentation
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential([
                      iaa.GaussianBlur(sigma=(0, 1.0)),
                      iaa.Fliplr(0.5),
                      iaa.Crop(percent=(0, 0.2)),
                      iaa.CoarseDropout(p=0.1, size_percent=(0.15, 0.3)),
                              sometimes(iaa.Affine(                                                                                                       
                                                    scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                                                    translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                                                    rotate=(-30, 30),
                                                    shear=(-16, 16)))
      ])

# Use Train Generator and Validation Genetator

train_datagen = ImageDataGenerator(
    rescale= 1./255,
    preprocessing_function=seq.augment_image
    )

valid_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', 
                                                    target_size=(img_height, img_width),
                                                    color_mode='rgb',
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True, seed=None)

Found 100000 images belonging to 200 classes.


In [0]:
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)

validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', 
                                                         x_col='File', y_col='Class',
                                                         target_size=(img_height, img_width),
                                                         color_mode='rgb', class_mode='categorical', 
                                                         batch_size=batch_size, shuffle=False, seed=None)    # Make shuffle false

def load_tiny_imagenet(path):
  
  # First load wnids
  wnids_file = os.path.join(path, 'wnids.txt')
  with open(os.path.join(wnids_file), 'r') as f:
    wnids = [x.strip() for x in f]
  wnids.sort()
  
  # Map wnids to integer labels
  wnid_to_label = {wnid: i for i, wnid in enumerate(wnids)}

  # Use words.txt to get names for each class
  words_file = os.path.join(path, 'words.txt')
  with open(os.path.join(words_file), 'r') as f:
    wnid_to_words = dict(line.split('\t') for line in f)
      
  return wnids, wnid_to_label, wnid_to_words

wnids, wnid_to_label, wnid_to_words = load_tiny_imagenet(path='tiny-imagenet-200/')
print('Training/Validation Data is created')

Found 10000 images belonging to 200 classes.
Training/Validation Data is created


In [0]:
X_train, y_train = next(train_generator)
X_test, y_test = next(validation_generator)

print("X_train Shape: ",X_train.shape)
print("y_train Shape: ",y_train.shape)
print("X_test Shape: ",X_test.shape)
print("y_test Shape: ",y_test.shape)

X_train Shape:  (128, 64, 64, 3)
y_train Shape:  (128, 200)
X_test Shape:  (128, 64, 64, 3)
y_test Shape:  (128, 200)


In [0]:
def bn_relu(input):
    norm = BatchNormalization()(input)
    return Activation("relu")(norm)
  
def conv_block(input_shape, filters, bottleneck, repetition=2, padding='same', non_residual=False):
  for i in range(repetition):
    if i == 0:
      activation = bn_relu(input_shape)
    else:
      activation = bn_relu(convolution)
    convolution = SeparableConv2D(filters=filters, kernel_size=(3,3), padding=padding, kernel_regularizer=regularizers.l2(0.01))(activation)      #kernel_regularizer=regularizers.l1(0.01)
    filters *= 2
  
  if non_residual==True:
    return Conv2D(filters=bottleneck, kernel_size=(1,1), activation='relu')(convolution)
  if input_shape.shape[3] != convolution.shape[3]:
    input_shape = Conv2D(filters=int(convolution.shape[3]), kernel_size=(1,1))(input_shape)
  
  concatenate = add([input_shape, convolution])
  
  if bottleneck==0:
    return concatenate
  else:
    output_shape = Conv2D(filters=bottleneck, kernel_size=(1,1), activation='relu')(concatenate)
    return output_shape

In [0]:
input = Input(shape=(None, None, channels))               # img_height, img_width

block1 = conv_block(input_shape=input, filters=32, bottleneck=0)
block2 = conv_block(input_shape=block1, filters=64, bottleneck=0)
block3 = conv_block(input_shape=block2, filters=128, bottleneck=128)

block4 = MaxPooling2D(pool_size=(2,2))(block3)

block5 = conv_block(input_shape=block4, filters=128, bottleneck=0)
block6 = conv_block(input_shape=block5, filters=256, bottleneck=0)

block7 = MaxPooling2D(pool_size=(2,2))(block6)

block8 = conv_block(input_shape=block7, filters=512, bottleneck=512)
block9 = conv_block(input_shape=block8, filters=1024, bottleneck=200, repetition=2, non_residual=True)

output = GlobalAveragePooling2D()(block9)
output = Activation('softmax')(output)

model = Model(inputs=input, outputs=output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 12          input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, None, None, 3 155         activation_1[0][0]               
__________________________________________________________________________________________________
batch_norm

In [0]:
model= load_model('gdrive/My Drive/EIP/take3_final.h5')           #  = load_model    .load_weights
print('Model/Weights from previous session loaded')

Model/Weights from previous session loaded


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

checkpoint = ModelCheckpoint('gdrive/My Drive/EIP/take3_final.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
lr_reducer = ReduceLROnPlateau(monitor='acc', mode='auto', factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
csv_logger = CSVLogger('gdrive/My Drive/EIP/resnet_tiny2.csv')

## Class weights

In [0]:
# Method 2:

y_pred = model.predict_generator(validation_generator, steps=79) # Steps 20 only if batch_size = 512
y_pred = np.argmax(y_pred, axis=1)
y_true = validation_generator.classes

actual_words = []
for i in val_data.Class:
  actual_words.append(wnid_to_words[i])
  
df = pd.DataFrame({'File Name': val_data.File, 'actual label': val_data.Class, 'actual words': actual_words, 'actual pred': y_true, 'predictions': y_pred}, 
                  columns=['File Name', 'actual label', 'actual words', 'actual pred', 'predictions'])
df.to_csv('gdrive/My Drive/EIP/prediction_classwise.csv')

df['correct'] = np.where(df['actual pred'] == df['predictions'], df['actual label'], 'Nan')
hard_df = df.loc[df['correct'] != 'Nan']

hard_df_count = hard_df.groupby(['actual pred']).size().reset_index(name='count')
dict_labels_vs_samples = hard_df_count.set_index('actual pred').T.to_dict('index').pop('count')

In [ ]:
# -------------------------------NOT REQUIRED------------------------------------
# Move Images if required to check the model is performing badly on a few datasets
import shutil
hard_images = hard_df['File Name'].tolist()
for f in hard_images:
  shutil.move('./tiny-imagenet-200/val/images/{}'.format(f), 'gdrive/My Drive/EIP/hard_classwise/{}'.format(f))

In [0]:
# -------------------------------ONLY FOR CLASS WEIGHTS--------------------------
def get_class_weights(dict_labels_vs_samples, balanced=True):
  
  if not len(dict_labels_vs_samples) == 200:
    print('WARNING: dict passed is NOT of length 200 - meaning not all 200 classes are included in the dict')
    
  keys = dict_labels_vs_samples.keys()
  values = list(dict_labels_vs_samples.values())
  total_samples = sum(values)
  num_classes = len(values)
  max_of_all_classes = max(values)
  average_of_all_classes = total_samples / num_classes
  multiplying_factor = 1
  
  if balanced:
    multiplying_factor = max_of_all_classes / average_of_all_classes
  
  print('total_samples: ', total_samples)
  print('num_classes: ', num_classes)
  print('max_of_all_classes: ', max_of_all_classes)
  print('multiplying_factor: ', multiplying_factor)
  
  class_weight = dict()

  for key in keys:
        num_correct_pred = int(dict_labels_vs_samples.get(key))
        if num_correct_pred == 0:
          num_correct_pred = 1 # this is to avoid divide by zero error, if a class has no correct predictions
        score_for_class = (total_samples / (num_classes * num_correct_pred)) * multiplying_factor
        class_weight[key] = score_for_class
        
  return class_weight

class_weights = get_class_weights(dict_labels_vs_samples)

total_samples:  5152
num_classes:  200
max_of_all_classes:  44
multiplying_factor:  1.7080745341614907


In [0]:
score = model.evaluate(X_test, y_test, verbose=1)
print('Test Accuracy: ', score[1])

128/128 [==============================] - 1s 11ms/step
Test Accuracy:  0.578125
